# Project (Section 7)

This is where we left off in the previous section project:

In [1]:
from datetime import date
from enum import Enum
from uuid import uuid4
from pydantic import BaseModel, ConfigDict, Field, field_serializer
from pydantic.alias_generators import to_camel
from pydantic import UUID4


class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"


class Automobile(BaseModel):
    model_config = ConfigDict(
        extra="forbid",
        str_strip_whitespace=True,
        validate_default=True,
        validate_assignment=True,
        alias_generator=to_camel,
    )

    id_: UUID4 | None = Field(alias="id", default_factory=uuid4) 
    manufacturer: str
    series_name: str
    type_: AutomobileType = Field(alias="type")
    is_electric: bool = False
    manufactured_date: date = Field(validation_alias="completionDate", ge=date(1980, 1, 1))
    base_msrp_usd: float = Field(
        validation_alias="msrpUSD", 
        serialization_alias="baseMSRPUSD"
    )
    vin: str
    number_of_doors: int = Field(
        default=4, 
        validation_alias="doors",
        ge=2,
        le=4,
        multiple_of=2,
    )
    registration_country: str | None = None
    license_plate: str | None = None

    @field_serializer("manufactured_date", when_used="json-unless-none")
    def serialize_date(self, value: date) -> str:
        return value.strftime("%Y/%m/%d")

Create an annotated type, named `BoundedString` to define a string that has a minimum of 2 characters, and no more than 50 characters.

Modify your `Automobile` model to use this annotated type for the following fields:
- `manufacturer`
- `series_name`
- `vin`
- `registration_country`
- `license_plate`

Create an annotated type, called `BoundedList` that uses a type variable to define a list of elements with a minimum of `1` element and a maximum of `5` elements.

Using this annotated type, add a new field to the model as follows:
- field name should be `top_features`
- place it just before the `vin` field
- it should both deserialize from and serialize to `topFeatures`
- if should be a bounded list of strings, which themselves shoudl be bounded to a minimum of `2` chars, and no more than `50`. (Hint: use the `BoundedString` type you create as the type when you define the field type in your model with `BoundedList`)
- make it optional, with a default of `None`

Use this data to test your model:

In [2]:
from uuid import UUID

data = {
    "id": "c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7",
    "manufacturer": "BMW",
    "seriesName": "M4 Competition xDrive",
    "type": "Convertible",
    "isElectric": False,
    "completionDate": "2023-01-01",
    "msrpUSD": 93_300,
    "topFeatures": ["6 cylinders", "all-wheel drive", "convertible"],
    "vin": "1234567890",
    "doors": 2,
    "registrationCountry": "France",
    "licensePlate": "AAA-BBB"
}

expected_serialized_by_alias = {
    'id': UUID('c4e60f4a-3c7f-4da5-9b3f-07aee50b23e7'),
    'manufacturer': 'BMW',
    'seriesName': 'M4 Competition xDrive',
    'type': AutomobileType.convertible,
    'isElectric': False,
    'manufacturedDate': date(2023, 1, 1),
    'baseMSRPUSD': 93300.0,
    'topFeatures': ['6 cylinders', 'all-wheel drive', 'convertible'],
    'vin': '1234567890',
    'numberOfDoors': 2,
    'registrationCountry': 'France',
    'licensePlate': 'AAA-BBB'
}

You should also test that you get validation errors if you attempt to deserialize data that does not conform to the constraints. 

> Suggestion: create simple single field models to just test your Annotated types and make sure they work as expected. Once you have done that, go ahead and use them in your main model.